# Imports

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:

CKPT = 'IG/tt100k-inception/frozen_inference_graph.pb'

PATH_TO_LABELS = 'data/label-tt100k.pbtxt'

NUM_CLASSES = 45

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()

with detection_graph.as_default():
    
  od_graph_def = tf.GraphDef()

  with tf.gfile.GFile(CKPT, 'rb') as fid:
        
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'name': 'class1', 'id': 1}, 2: {'name': 'class2', 'id': 2}, 3: {'name': 'class3', 'id': 3}, 4: {'name': 'class4', 'id': 4}, 5: {'name': 'class5', 'id': 5}, 6: {'name': 'class6', 'id': 6}, 7: {'name': 'class7', 'id': 7}, 8: {'name': 'class8', 'id': 8}, 9: {'name': 'class9', 'id': 9}, 10: {'name': 'class10', 'id': 10}, 11: {'name': 'class11', 'id': 11}, 12: {'name': 'class12', 'id': 12}, 13: {'name': 'class13', 'id': 13}, 14: {'name': 'class14', 'id': 14}, 15: {'name': 'class15', 'id': 15}, 16: {'name': 'class16', 'id': 16}, 17: {'name': 'class17', 'id': 17}, 18: {'name': 'class18', 'id': 18}, 19: {'name': 'class19', 'id': 19}, 20: {'name': 'class20', 'id': 20}, 21: {'name': 'class21', 'id': 21}, 22: {'name': 'class22', 'id': 22}, 23: {'name': 'class23', 'id': 23}, 24: {'name': 'class24', 'id': 24}, 25: {'name': 'class25', 'id': 25}, 26: {'name': 'class26', 'id': 26}, 27: {'name': 'class27', 'id': 27}, 28: {'name': 'class28', 'id': 28}, 29: {'name': 'class29', 'id': 29}, 30: {'name'

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
from glob import glob

In [20]:

PATH_TO_TEST_IMAGES_DIR = '/media/nirmal/DATA2/Final-Year-Project/Dataset/Traffic Sign/Tsinghua Tencent 100k/data/test-1024/part-1/'

#print(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))
#TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))  # PNG OR JPG
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))
print("Length of test images:", len(TEST_IMAGE_PATHS))

print(TEST_IMAGE_PATHS[45][104:-4])
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

Length of test images: 3071
15641


In [21]:
import time
import json
counter = 0
with detection_graph.as_default():
    time0 = time.time()
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            counter = counter + 1
            print('Image {}'.format(counter))
            filename_string = image_path[104:-4]
            image = Image.open(image_path)
            # print(image.size)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)

            #time0 = time.time()

            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})

            

            # Visualization of the results of a detection.
            # vis_util.visualize_boxes_and_labels_on_image_array(
            #  image_np,
            #  np.squeeze(boxes),
            #  np.squeeze(classes).astype(np.int32),
            #  np.squeeze(scores),
            #  category_index,
            #  use_normalized_coordinates=True,
            #  line_thickness=8)
            #plt.figure(figsize=IMAGE_SIZE)
            #plt.imshow(image_np)
            #plt.show()
            coordinates = vis_util.return_coordinates(
                        image,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=8,
                        min_score_thresh=0.50)
            #textfile = open("json/"+filename_string+".json", "a")
            #textfile.write(json.dumps(coordinates))
            #textfile.write("\n")
            #print(coordinates)
            if coordinates == []:
                with open('test-detections-tt100k/inception/' + filename_string + '.txt','a') as text_file:
                    text_file.close()
                
            for coordinate in coordinates:
                #print(coordinate)
                (xmax, xmin, ymax, ymin, accuracy, classification_num) = coordinate
                label = classification_num
                detection_box = (label, accuracy, xmin, ymin, xmax, ymax)
                
                #if label == 46:
                #    light = 'Red'
                    
                #if label == 47:
                #    light = 'Yellow'
                    
                #if label == 48:
                #    light = 'Green'
                    
                #if label == 49:
                #    light = 'off'
                    
                #print('{} {} {} {} {} {}'.format(label, accuracy, xmin, ymin, xmax, ymax))
                
                #if label == 46 or label == 47 or label == 48 or label == 49: 
                with open('test-detections-tt100k/inception/' + filename_string + '.txt','a') as text_file:
                    text_file.write(str(label) + ' ' + str(accuracy) + ' ' + str(xmin) + ' ' + str(ymin) + ' ' + str(xmax) + ' ' + str(ymax) + '\n')
                    text_file.close()    
                
                
                
                #print('{} {} {} {} {} {}'.format(label, accuracy, xmin, ymin, xmax, ymax))
                print(detection_box)
                #if str(label) != '46' and str(label) != '47' and str(label) != '48' and str(label) != '49':
                    
                        
            time1 = time.time()

            print("Time in milliseconds", (time1 - time0) * 1000) 
                
             

Image 1
(2, 0.613962, 610, 463, 620, 476)
Time in milliseconds 19301.01490020752
Image 2
Time in milliseconds 21025.59757232666
Image 3
Time in milliseconds 23102.214336395264
Image 4
(1, 0.9983375, 791, 277, 838, 331)
(4, 0.82920307, 734, 270, 787, 326)
Time in milliseconds 25137.322664260864
Image 5
Time in milliseconds 27231.966733932495
Image 6
Time in milliseconds 28968.466758728027
Image 7
Time in milliseconds 30513.667345046997
Image 8
Time in milliseconds 32184.952974319458
Image 9
(1, 0.9768702, 965, 410, 981, 430)
(1, 0.61781234, 961, 408, 977, 430)
(1, 0.81587285, 973, 411, 986, 432)
Time in milliseconds 33890.960931777954
Image 10
(1, 0.9753135, 807, 184, 879, 263)
(4, 0.69345045, 884, 202, 937, 273)
Time in milliseconds 35829.91576194763
Image 11
Time in milliseconds 37558.9964389801
Image 12
Time in milliseconds 39436.78021430969
Image 13
Time in milliseconds 41137.82024383545
Image 14
(2, 0.99201125, 662, 396, 688, 424)
(1, 0.71715045, 789, 452, 801, 465)
(3, 0.98786575,

(15, 0.6645967, 655, 186, 705, 230)
(15, 0.5140908, 460, 186, 519, 230)
(13, 0.5348926, 224, 222, 263, 262)
(13, 0.5408262, 391, 195, 439, 235)
Time in milliseconds 183739.22872543335
Image 103
Time in milliseconds 185396.13604545593
Image 104
(3, 0.97790635, 53, 230, 97, 285)
(1, 0.80301607, 609, 429, 625, 446)
(1, 0.9466226, 616, 430, 630, 445)
Time in milliseconds 186854.34436798096
Image 105
(3, 0.7355056, 144, 440, 154, 454)
(3, 0.57809496, 136, 441, 148, 454)
Time in milliseconds 188333.37712287903
Image 106
Time in milliseconds 189801.41377449036
Image 107
(1, 0.60757184, 427, 477, 437, 489)
Time in milliseconds 191308.19749832153
Image 108
Time in milliseconds 192780.08008003235
Image 109
(4, 0.64858615, 860, 372, 895, 409)
Time in milliseconds 194232.5656414032
Image 110
(2, 0.96202135, 891, 548, 915, 577)
Time in milliseconds 195891.11590385437
Image 111
(1, 0.98701197, 933, 372, 955, 398)
(9, 0.829083, 688, 428, 723, 464)
(9, 0.9266565, 689, 470, 722, 504)
Time in millisecon

Time in milliseconds 324574.5940208435
Image 198
(1, 0.96040386, 23, 69, 105, 171)
(4, 0.8637129, 110, 116, 170, 208)
Time in milliseconds 326009.69433784485
Image 199
(1, 0.66606325, 106, 417, 122, 440)
Time in milliseconds 327452.47888565063
Image 200
Time in milliseconds 328905.82728385925
Image 201
(2, 0.96340954, 993, 427, 1007, 445)
Time in milliseconds 330341.24970436096
Image 202
Time in milliseconds 331806.449174881
Image 203
Time in milliseconds 333235.23354530334
Image 204
(1, 0.9659621, 175, 389, 193, 413)
(2, 0.6356346, 641, 230, 663, 256)
Time in milliseconds 334712.7516269684
Image 205
(2, 0.992392, 653, 427, 671, 447)
(3, 0.98350614, 722, 429, 739, 446)
Time in milliseconds 336162.0545387268
Image 206
(1, 0.95087844, 12, 413, 44, 477)
(9, 0.5162933, 8, 343, 41, 411)
(9, 0.75902253, 42, 361, 85, 410)
Time in milliseconds 337605.63349723816
Image 207
(2, 0.528094, 513, 435, 522, 447)
Time in milliseconds 339075.1202106476
Image 208
Time in milliseconds 340552.4344444275
I

Time in milliseconds 472397.54605293274
Image 299
Time in milliseconds 473854.44808006287
Image 300
Time in milliseconds 475307.66105651855
Image 301
(2, 0.9541669, 835, 476, 856, 501)
Time in milliseconds 476750.6654262543
Image 302
Time in milliseconds 478201.1842727661
Image 303
Time in milliseconds 479656.51416778564
Image 304
(1, 0.9969297, 804, 490, 833, 520)
Time in milliseconds 481136.6574764252
Image 305
(15, 0.7362972, 802, 423, 820, 440)
(13, 0.61259973, 699, 420, 717, 436)
Time in milliseconds 482615.7920360565
Image 306
Time in milliseconds 484092.69404411316
Image 307
Time in milliseconds 485549.92055892944
Image 308
(27, 0.8669464, 402, 485, 436, 520)
Time in milliseconds 486996.9370365143
Image 309
(2, 0.6537673, 418, 472, 427, 482)
Time in milliseconds 488442.0323371887
Image 310
Time in milliseconds 489923.85506629944
Image 311
(2, 0.99122345, 299, 419, 319, 442)
(1, 0.542672, 550, 455, 563, 468)
(1, 0.5694248, 554, 451, 567, 468)
(3, 0.97667754, 377, 414, 400, 438)
T

Time in milliseconds 618975.1822948456
Image 399
Time in milliseconds 620415.6696796417
Image 400
(1, 0.90300035, 658, 442, 669, 455)
(1, 0.61542064, 665, 442, 676, 455)
(24, 0.6205629, 667, 549, 701, 582)
(1, 0.7702039, 654, 442, 666, 455)
Time in milliseconds 621870.2108860016
Image 401
Time in milliseconds 623343.2078361511
Image 402
(19, 0.5065533, 198, 468, 231, 507)
Time in milliseconds 624800.2846240997
Image 403
(13, 0.7758177, 974, 347, 994, 372)
(15, 0.73407274, 993, 346, 1016, 377)
Time in milliseconds 626234.5652580261
Image 404
(13, 0.6672009, 783, 370, 804, 393)
(15, 0.5673964, 868, 377, 891, 403)
(13, 0.619745, 923, 385, 940, 410)
(13, 0.6597044, 856, 377, 875, 401)
Time in milliseconds 627698.2009410858
Image 405
(2, 0.69205874, 495, 406, 507, 421)
Time in milliseconds 629138.991355896
Image 406
Time in milliseconds 630576.39503479
Image 407
(2, 0.9774375, 711, 425, 742, 455)
(1, 0.5006517, 598, 444, 610, 458)
(1, 0.5903703, 584, 444, 598, 459)
(3, 0.6249861, 811, 486, 

...

Time in milliseconds 3924569.899082184
Image 2648
Time in milliseconds 3926011.105775833
Image 2649
Time in milliseconds 3927466.2051200867
Image 2650
(1, 0.5414553, 714, 442, 726, 455)
Time in milliseconds 3928945.0595378876
Image 2651
(3, 0.9929796, 132, 507, 158, 541)
Time in milliseconds 3930376.160144806
Image 2652
(1, 0.8561565, 33, 498, 46, 519)
Time in milliseconds 3931825.0188827515
Image 2653
Time in milliseconds 3933277.7333259583
Image 2654
Time in milliseconds 3934739.6705150604
Image 2655
(1, 0.71161205, 223, 459, 244, 482)
Time in milliseconds 3936199.1860866547
Image 2656
Time in milliseconds 3937650.6938934326
Image 2657
Time in milliseconds 3939107.3083877563
Image 2658
Time in milliseconds 3940565.440893173
Image 2659
(19, 0.5472185, 700, 489, 732, 524)
Time in milliseconds 3942002.729177475
Image 2660
(1, 0.97772616, 669, 436, 682, 450)
(1, 0.9193813, 664, 436, 678, 450)
Time in milliseconds 3943464.5261764526
Image 2661
(2, 0.9880674, 219, 402, 245, 430)
(1, 0.7272

Time in milliseconds 4067561.465024948
Image 2746
Time in milliseconds 4069036.885738373
Image 2747
(2, 0.9941147, 239, 412, 260, 437)
(1, 0.58750606, 567, 439, 583, 457)
(1, 0.588287, 590, 439, 606, 458)
(3, 0.99292034, 323, 406, 344, 429)
Time in milliseconds 4070490.0376796722
Image 2748
Time in milliseconds 4071937.306880951
Image 2749
(19, 0.71933776, 800, 468, 826, 527)
Time in milliseconds 4073372.711658478
Image 2750
Time in milliseconds 4074830.6839466095
Image 2751
(1, 0.955083, 253, 26, 300, 124)
Time in milliseconds 4076261.9411945343
Image 2752
(2, 0.9948872, 83, 364, 98, 392)
(2, 0.503695, 580, 440, 593, 455)
(3, 0.92459476, 657, 443, 670, 460)
Time in milliseconds 4077726.5939712524
Image 2753
Time in milliseconds 4079151.697874069
Image 2754
Time in milliseconds 4080588.538646698
Image 2755
Time in milliseconds 4082020.761013031
Image 2756
(1, 0.9968893, 705, 381, 739, 419)
Time in milliseconds 4083490.622997284
Image 2757
Time in milliseconds 4084931.2143325806
Image 2

Time in milliseconds 4209671.972513199
Image 2843
Time in milliseconds 4211162.519693375
Image 2844
(2, 0.97846895, 657, 426, 674, 444)
(2, 0.5348973, 661, 426, 678, 445)
(3, 0.94504344, 704, 427, 722, 446)
(1, 0.947889, 112, 419, 123, 436)
(1, 0.94817215, 835, 443, 845, 454)
Time in milliseconds 4212648.132801056
Image 2845
(15, 0.7300574, 936, 364, 956, 392)
(13, 0.56136745, 921, 363, 938, 390)
(13, 0.67316604, 854, 355, 877, 381)
(13, 0.7109345, 782, 346, 806, 373)
Time in milliseconds 4214114.485025406
Image 2846
Time in milliseconds 4215584.977865219
Image 2847
(27, 0.60936016, 734, 465, 789, 524)
Time in milliseconds 4217020.822286606
Image 2848
(2, 0.9796562, 613, 450, 626, 463)
(2, 0.6479679, 609, 450, 622, 463)
(2, 0.8908553, 617, 450, 628, 462)
Time in milliseconds 4218498.609781265
Image 2849
Time in milliseconds 4219983.596086502
Image 2850
(2, 0.9623676, 596, 447, 607, 459)
(2, 0.8949929, 594, 447, 604, 459)
(3, 0.9584238, 661, 448, 672, 462)
Time in milliseconds 4221453.3

Time in milliseconds 4342966.818332672
Image 2934
(4, 0.557567, 414, 282, 462, 338)
Time in milliseconds 4344399.224758148
Image 2935
Time in milliseconds 4345849.010944366
Image 2936
(1, 0.9693123, 184, 348, 205, 384)
(9, 0.7839866, 558, 458, 573, 474)
(1, 0.9378363, 996, 405, 1016, 427)
Time in milliseconds 4347319.238185883
Image 2937
Time in milliseconds 4348754.470348358
Image 2938
Time in milliseconds 4350189.750432968
Image 2939
(27, 0.61595905, 668, 412, 703, 443)
(21, 0.564715, 850, 478, 872, 500)
Time in milliseconds 4351643.716573715
Image 2940
Time in milliseconds 4353081.997394562
Image 2941
(2, 0.7584297, 496, 344, 532, 381)
(1, 0.60398775, 706, 433, 720, 449)
Time in milliseconds 4354530.068635941
Image 2942
Time in milliseconds 4355979.640245438
Image 2943
(2, 0.89484775, 473, 452, 483, 464)
Time in milliseconds 4357440.933227539
Image 2944
(1, 0.992183, 665, 396, 695, 425)
(2, 0.95124143, 737, 264, 761, 289)
Time in milliseconds 4358909.306764603
Image 2945
(1, 0.60389

(12, 0.7785659, 606, 291, 650, 338)
(11, 0.5761519, 552, 295, 603, 339)
Time in milliseconds 4479847.458362579
Image 3028
(2, 0.9985812, 388, 454, 418, 486)
(1, 0.6545638, 630, 460, 639, 470)
(1, 0.82982486, 627, 460, 637, 470)
(2, 0.98441523, 102, 463, 132, 492)
Time in milliseconds 4481325.777769089
Image 3029
Time in milliseconds 4482782.858848572
Image 3030
Time in milliseconds 4484243.351697922
Image 3031
Time in milliseconds 4485722.84078598
Image 3032
Time in milliseconds 4487303.742408752
Image 3033
(19, 0.5405325, 834, 427, 858, 474)
Time in milliseconds 4488751.941919327
Image 3034
Time in milliseconds 4490219.03014183
Image 3035
(25, 0.8862601, 141, 380, 170, 424)
Time in milliseconds 4491707.621097565
Image 3036
(2, 0.9595991, 347, 424, 361, 441)
Time in milliseconds 4493177.0169734955
Image 3037
(1, 0.66722304, 603, 465, 616, 479)
Time in milliseconds 4494651.482343674
Image 3038
Time in milliseconds 4496123.2097148895
Image 3039
(1, 0.9839909, 112, 317, 133, 364)
Time in 